In [1]:
# Wrangling Jupyter Notebook (wrangling.ipynb)

# This notebook cleans and wrangles the raw CWUR university rankings dataset.
# It handles missing data, converts types, creates features, and normalizes scores.

import pandas as pd
import numpy as np

# Load raw data

try:
    df = pd.read_csv("cwur_rankings_raw.csv")
    print(f"Loaded dataset successfully: {len(df)} rows, {len(df.columns)} columns")
except FileNotFoundError:
    # Stop execution if the raw data file is missing
    raise SystemExit("File 'cwur_rankings_raw.csv' not found. Run acquisition first!")

# Preview first few rows of the raw dataset
print("\nSample of raw data:")
print(df.head())

# Handle missing data

# Identify missing values in 'Rank' and 'Score' columns
missing_rank = df["Rank"].isna().sum()
missing_score = df["Score"].isna().sum()
print(f"\nMissing Ranks: {missing_rank}, Missing Scores: {missing_score}")

# Justification for handling missing data:
# - Rank is critical for analysis, so rows without rank are removed.
# - Score is optional; missing values will be replaced with 0 (interpreted as not scored).
df = df.dropna(subset=["Rank"])  # remove rows with no rank
df["Score"] = df["Score"].fillna("0")  # replace missing scores with 0


# Data type conversion

def clean_numeric(value):
    """
    Convert rank or score text to numeric float safely.
    Handles:
    - Ranges like '101-150' (converts to midpoint)
    - Non-numeric symbols like 'N/A', '†', '%'
    - Commas in numbers
    """
    if pd.isna(value):
        return np.nan
    value = str(value).replace(",", "").strip()
   
    # Handle ranges like "101-150"
    if "-" in value:
        parts = value.split("-")
        try:
            value = (float(parts[0]) + float(parts[1])) / 2
        except:
            return np.nan
   
    # Remove non-numeric symbols
    value = value.replace("N/A", "").replace("†", "").replace("%", "")
   
    try:
        return float(value)
    except ValueError:
        return np.nan

# Apply the cleaning function to 'Rank' and 'Score'

df["Rank"] = df["Rank"].apply(clean_numeric).astype("Int64", errors="ignore")
df["Score"] = df["Score"].apply(clean_numeric)

print("\nConverted Rank and Score to numeric types:")
print(df.dtypes)

# Feature engineering

# Map countries to broader global regions for analytics

region_map = {
    # North America
    "United States": "North America",
    "Canada": "North America",
    "Mexico": "North America",

    # Europe
    "United Kingdom": "Europe",
    "Germany": "Europe",
    "France": "Europe",
    "Italy": "Europe",
    "Spain": "Europe",
    "Netherlands": "Europe",
    "Switzerland": "Europe",
    "Sweden": "Europe",
    "Denmark": "Europe",
    "Finland": "Europe",
    "Norway": "Europe",
    "Belgium": "Europe",
    "Austria": "Europe",
    "Russia": "Europe",

    # Asia
    "China": "Asia",
    "Japan": "Asia",
    "South Korea": "Asia",
    "India": "Asia",
    "Singapore": "Asia",
    "Hong Kong": "Asia",
    "Taiwan": "Asia",
    "Malaysia": "Asia",
    "Thailand": "Asia",
    "Saudi Arabia": "Asia",
    "Iran": "Asia",
    "Turkey": "Asia",

    # Oceania
    "Australia": "Oceania",
    "New Zealand": "Oceania",

    # Africa
    "South Africa": "Africa",
    "Egypt": "Africa",
    "Nigeria": "Africa",
    "Kenya": "Africa",
    "Uganda": "Africa",

    # South America
    "Brazil": "South America",
    "Argentina": "South America",
    "Chile": "South America",
    "Colombia": "South America"
}

# Apply region mapping, fill any unmatched country as "Other"

df["Global_Region"] = df["Country"].map(region_map).fillna("Other")

# Normalize the 'Score' column to a 0–1 range for comparability

if df["Score"].notna().any():
    min_score = df["Score"].min()
    max_score = df["Score"].max()
    df["Overall_Score_Normalized"] = (df["Score"] - min_score) / (max_score - min_score)
else:
    df["Overall_Score_Normalized"] = np.nan


# Export cleaned data

df.to_csv("cwur_rankings_cleaned.csv", index=False)

print("\nData wrangling completed successfully!")
print("Cleaned data saved as 'cwur_rankings_cleaned.csv'")

# Preview the first 10 rows of cleaned data
print("\nCleaned Data Preview:")
print(df.head(10))

Loaded dataset successfully: 10000 rows, 4 columns

Sample of raw data:
        Rank                             University         Country  Score
0  1Top 0.1%                     Harvard University             USA      1
1  2Top 0.1%  Massachusetts Institute of Technology             USA      2
2  3Top 0.1%                    Stanford University             USA      3
3  4Top 0.1%                University of Cambridge  United Kingdom      1
4  5Top 0.1%                   University of Oxford  United Kingdom      2

Missing Ranks: 0, Missing Scores: 0

Converted Rank and Score to numeric types:
Rank            Int64
University     object
Country        object
Score         float64
dtype: object

Data wrangling completed successfully!
Cleaned data saved as 'cwur_rankings_cleaned.csv'

Cleaned Data Preview:
   Rank                             University         Country  Score  \
0  <NA>                     Harvard University             USA    1.0   
1  <NA>  Massachusetts Institute of 